**Imports**

In [99]:
import os
import glob
import pandas as pd
import SimpleITK as sitk
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

**Data preparation**

In [44]:
DEBUG = True

In [45]:
def parse_info_file(info_file_path):
    info_dict = {}
    with open(info_file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split(': ')
            info_dict[key] = value if key == 'Group' else float(value) if '.' in value else int(value)
    return info_dict


def get_acdc_file_paths(data_paths):
    data_records = []
    for data_path, dataset_type in data_paths:
        # Get all the image and ground truth file paths
        image_pattern = os.path.join(data_path, 'patient*/patient*_frame??.nii.gz')
        gt_pattern = os.path.join(data_path, 'patient*/patient*_frame??_gt.nii.gz')
        image_paths = glob.glob(image_pattern)
        gt_paths = glob.glob(gt_pattern)

        # Convert gt_paths to a dictionary for quick lookup
        gt_dict = {os.path.basename(gt_path).replace('_gt', ''): gt_path for gt_path in gt_paths}

        for image_path in image_paths:
            basename = os.path.basename(image_path)
            patient_number = basename.split('_')[0].replace('patient', '')
            frame_number = basename.split('_')[1].replace('frame', '').replace('.nii.gz', '')

            # Extracting the directory path to read the info.cfg
            patient_dir = os.path.dirname(image_path)
            info_file_path = os.path.join(patient_dir, 'Info.cfg')
            patient_info = parse_info_file(info_file_path)

            gt_path = gt_dict.get(basename, None)

            data_records.append({
                'patient_number': patient_number,
                'frame_number': frame_number,
                'image_path': image_path,
                'gt_path': gt_path,
                'dataset_type': dataset_type,
                **patient_info  # Add the info.cfg data to the record
            })

    # Create a DataFrame
    df = pd.DataFrame(data_records)

    return df

**Validate assumptions**

Before the dataframe is processed, we validate some assumptions that would make the process easier:

1. The rows are ordered such that for i = 0,2,4,...,288: rows i and i+1 always belong to patient_number i

2. The rows are ordered such that for i = 0,2,4,...,288: the frame number of row i always matches the ED number, and the frame number of row i+1 always matches the ES number, ignoring any leading zeros (ie: 01 = 1)

In [46]:
# Call the function with both paths and their dataset types
data_paths = [('../data/training', 'Training'), ('../data/testing', 'Testing')]
df_metadata = get_acdc_file_paths(data_paths)


def validate_dataframe(df):
    # Check for assumptions
    valid = True
    messages = []

    # Iterate over the DataFrame in steps of 2
    for i in range(0, len(df), 2):
        # Ensure there are enough rows remaining
        if i + 1 >= len(df):
            break

        row_current = df.iloc[i]
        row_next = df.iloc[i + 1]

        # Assumption 1: Rows i and i+1 belong to the same patient_number
        if row_current['patient_number'] != row_next['patient_number']:
            valid = False
            messages.append(f"Rows {i} and {i+1} do not belong to the same patient_number.")

        # Assumption 2: Frame number of row i matches ED,
        # and frame number of row i+1 matches ES (ignoring leading zeros)
        frame_number_current = int(row_current['frame_number'])
        frame_number_next = int(row_next['frame_number'])
        ed_number = row_current['ED']
        es_number = row_next['ES']

        if frame_number_current != ed_number:
            valid = False
            messages.append(f"Row {i} frame_number {frame_number_current} does not match ED {ed_number}.")

        if frame_number_next != es_number:
            valid = False
            messages.append(f"Row {i+1} frame_number {frame_number_next} does not match ES {es_number}.")

    return valid, messages


valid, messages = validate_dataframe(df_metadata)

if valid:
    print("The DataFrame meets all the specified assumptions.")
else:
    print("The DataFrame does not meet the specified assumptions:")
    for message in messages:
        print(message)

The DataFrame meets all the specified assumptions.


**Feature extraction methods**

In [66]:
def get_voxel_volume(image):
    spacing = image.GetSpacing()
    vol_voxel = spacing[0] * spacing[1] * spacing[2]
    return vol_voxel


def extract_features(image):
    vol_voxel = get_voxel_volume(image)

    # Convert the image to a NumPy array
    img_array = sitk.GetArrayFromImage(image)

    num_RV = np.sum(img_array == 1)
    num_MY = np.sum(img_array == 2)
    num_LV = np.sum(img_array == 3)

    vol_RV = num_RV * (vol_voxel / 1000)
    vol_MY = num_MY * (vol_voxel / 1000)
    vol_LV = num_LV * (vol_voxel / 1000)
    
    mass_MY = vol_MY * 1.055; # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6461811/

    return vol_RV, vol_MY, vol_LV, mass_MY

**Features**

Volumes in ml of:

1. vRVED: right ventricle cavivity (RV) during End Dyastolic (ED)

2. vLVED: left ventricle cavivity (LV) during ED

3. vMYED: myocardium (MY) during ED

4. vRVES: RV during during End Systolic (ES)

5. vLVES: LV during ES

6. vMYES: MY during ES

In [74]:
df_metadata_copy = df_metadata.copy()

# Extract features
features_list = []
for path in df_metadata['gt_path']:
    segmentation_mask = sitk.ReadImage(path)
    if segmentation_mask is not None:
        features = extract_features(segmentation_mask)
        features_list.append(features)

# Convert the list of tuples to a NumPy array for better performance
features_list = np.array(features_list)

df_metadata_copy[['vRV', 'vMY', 'vLV', 'mMY']] = features_list

print(df_metadata_copy.head(5))

NiftiImageIO (0x17e52d0): ../data/training/patient001/patient001_frame01_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x17e52d0): ../data/training/patient001/patient001_frame01_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x1857ae0): ../data/training/patient001/patient001_frame12_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x1857ae0): ../data/training/patient001/patient001_frame12_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x17e52d0): ../data/training/patient002/patient002_frame01_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x17e52d0): ../data/training/patient002/patient002_frame01_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x1857ae0): ../data/training/patient002/patient002_frame12_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x1857ae0): ../data/training/patient002/patient002_frame12_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x17e52d0): ../data/training/patient003/patient003_frame01_gt.nii.

  patient_number frame_number  \
0            001           01   
1            001           12   
2            002           01   
3            002           12   
4            003           01   

                                          image_path  \
0  ../data/training/patient001/patient001_frame01...   
1  ../data/training/patient001/patient001_frame12...   
2  ../data/training/patient002/patient002_frame01...   
3  ../data/training/patient002/patient002_frame12...   
4  ../data/training/patient003/patient003_frame01...   

                                             gt_path dataset_type  ED  ES  \
0  ../data/training/patient001/patient001_frame01...     Training   1  12   
1  ../data/training/patient001/patient001_frame12...     Training   1  12   
2  ../data/training/patient002/patient002_frame01...     Training   1  12   
3  ../data/training/patient002/patient002_frame12...     Training   1  12   
4  ../data/training/patient003/patient003_frame01...     Training   1  15   

 

NiftiImageIO (0x17e52d0): ../data/testing/patient145/patient145_frame01_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x17e52d0): ../data/testing/patient145/patient145_frame01_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x1857ae0): ../data/testing/patient145/patient145_frame13_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x1857ae0): ../data/testing/patient145/patient145_frame13_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x17e52d0): ../data/testing/patient146/patient146_frame01_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x17e52d0): ../data/testing/patient146/patient146_frame01_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x1857ae0): ../data/testing/patient146/patient146_frame10_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x1857ae0): ../data/testing/patient146/patient146_frame10_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x17e52d0): ../data/testing/patient147/patient147_frame01_gt.nii.gz has un

In [98]:
unused_columns = ['image_path', 'gt_path', 'NbFrame', 'frame_number', 'ED', 'ES']
df_features = df_metadata_copy.drop(columns=unused_columns)

df_features_ED = df_features.groupby('patient_number').nth(0).reset_index()
df_features_ED.rename(columns={
    'vRV': 'vRVED',
    'vLV': 'vLVED',
    'vMY': 'vMYED',
    'mMY': 'mMYED'
}, inplace=True)

df_features_ES = df_features.groupby('patient_number').nth(1).reset_index()
df_features_ES.rename(columns={
    'vRV': 'vRVES',
    'vLV': 'vLVES',
    'vMY': 'vMYES',
    'mMY': 'mMYES'
}, inplace=True)

common_columns = ['Height', 'Weight', 'Group', 'dataset_type', 'index']
df_features_ES = df_features_ES.drop(columns=common_columns)

df_features = pd.merge(df_features_ED, df_features_ES, on='patient_number')

unused_columns = ['mMYES', 'vMYED', 'Height', 'Weight']
df_features = df_features.drop(columns=unused_columns)

df_features['efRV'] = (df_features['vRVED']-df_features['vRVES'])/df_features['vRVED']
df_features['efLV'] = (df_features['vLVED']-df_features['vLVES'])/df_features['vLVED']

df_training = df_features[df_features['dataset_type'] == 'Training']
df_training = df_training.drop(columns=['dataset_type', 'index', 'patient_number'])

df_testing = df_features[df_features['dataset_type'] == 'Testing']
dt_testing = df_testing.drop(columns=['dataset_type', 'index', 'patient_number'])

features = ['vRVED', 'vLVED', 'mMYED', 'vRVES', 'vMYES', 'vLVES', 'efRV', 'efLV']

print(df_training.head(5))

  Group       vRVED       vLVED       mMYED       vRVES       vMYES  \
0   DCM  139.721680  295.507812  173.291992   59.545898  195.068359   
1   DCM   94.432068  265.744400  169.257425   28.823090  192.565155   
2   DCM  192.333984  276.708984  202.397217  174.584961  201.074219   
3   DCM  106.264114  260.847092  177.717339   84.543991  174.957275   
4   DCM  170.463867  290.797119  212.469170   74.553223  232.261963   

        vLVES      efRV      efLV  
0  225.610352  0.573825  0.236533  
1  188.303375  0.694774  0.291412  
2  241.088867  0.092282  0.128728  
3  226.472473  0.204398  0.131781  
4  224.094727  0.562645  0.229378  


In [100]:
df_training = df_features[df_features['dataset_type'] == 'Training']
df_testing = df_features[df_features['dataset_type'] == 'Testing']

X_train = df_training[features]

X_test = df_testing[features]

y_train = df_training['Group']
y_test = df_testing['Group']

scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

# Creating and training the Random Forest model
rf_model = RandomForestClassifier(random_state=55)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

In [101]:
# Evaluating performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Evaluate recall and precision for each group/label separately and give the total f1-score
# Evaluating recall, precision, and F1-score for each group/label
report = classification_report(y_test, y_pred, output_dict=True)

# Printing the classification report
print("Classification Report:")
for label, metrics in report.items():
    if label in y_test.unique():
        print(f"Label: {label}")
        print(f"  Precision: {metrics['precision']:.4f}")
        print(f"  Recall: {metrics['recall']:.4f}")
        print(f"  F1-score: {metrics['f1-score']:.4f}")

# Printing the overall F1-score
print(f"\nOverall F1-score: {report['weighted avg']['f1-score']:.4f}")

Accuracy: 0.86
Classification Report:
Label: DCM
  Precision: 0.7273
  Recall: 0.8000
  F1-score: 0.7619
Label: HCM
  Precision: 1.0000
  Recall: 0.9000
  F1-score: 0.9474
Label: MINF
  Precision: 0.7778
  Recall: 0.7000
  F1-score: 0.7368
Label: NOR
  Precision: 0.9000
  Recall: 0.9000
  F1-score: 0.9000
Label: RV
  Precision: 0.9091
  Recall: 1.0000
  F1-score: 0.9524

Overall F1-score: 0.8597
